### Import the necessary packages

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

import warnings
warnings.filterwarnings('ignore')

### Read the datasets

In [2]:
data=pd.read_csv('menu_items.csv')

In [3]:
data_one=pd.read_csv('order_details.csv')

### Copying data to another varaible to avoid any changes to original data

In [4]:

menu = data.copy

In [5]:
order = data_one.copy

### Convert to a DataFrame if needed

In [6]:
menu = pd.DataFrame(data) 

In [7]:
order = pd.DataFrame(data_one)  

### Create database connection

In [8]:
connection = sqlite3.connect('Restaurant_orders.db')

### Insert data into database

In [9]:
menu.to_sql('menu', connection, 
if_exists = 'replace', index = False)

In [10]:
order.to_sql('order', connection, 
if_exists = 'replace', index = False)

# Objective 1
## Explore the menu items table
### Your first objective is to better understand the items table by finding the number of rows in the table, the least and most expensive items, and the item prices within each category.

### Run SQL query and load results into Dataframe

In [11]:

query = """ 
SELECT *
FROM menu
LIMIT 10;
"""

menu = pd.read_sql_query(query, connection)

menu

,menu_item_id,item_name,category,price
0,101,Hamburger,American,12.95
1,102,Cheeseburger,American,13.95
2,103,Hot Dog,American,9.00
3,104,Veggie Burger,American,10.50
4,105,Mac & Cheese,American,7.00
5,106,French Fries,American,7.00
6,107,Orange Chicken,Asian,16.50
7,108,Tofu Pad Thai,Asian,14.50
8,109,Korean Beef Bowl,Asian,17.95
9,110,Pork Ramen,Asian,17.95


#### There are 4 columns: menu_item_id (unique id of a menu item), item_name (a item on the menu), category (type of cuisine) and price (menu item cost in US dollars) 

### Run SQL query that counts the number of rows in the menu items table

In [12]:

query = """ 
SELECT COUNT(menu_item_id)
FROM menu;
"""
menu = pd.read_sql_query(query, connection)

menu

,COUNT(menu_item_id)
0,32


#### 32 rows for the menu table

### Let's find the distinct values in item_name

In [13]:
query = """ 
SELECT COUNT(DISTINCT(item_name))
FROM menu;
"""

menu = pd.read_sql_query(query, connection)

menu

,COUNT(DISTINCT(item_name))
0,32


#### There are 32 unique dishes in the table, equivalent to the total number of rows. This indicates that there are no duplicate values in this column.

### Let's write a query to find the number of null values for item_name and category in the menu table

In [14]:

query = """ 
SELECT *
FROM menu
WHERE item_name IS NULL OR category IS NULL;
"""

menu = pd.read_sql_query(query, connection)

menu



,menu_item_id,item_name,category,price


#### There are no null values for this table

### Now we will check if there are any values under 0 in the price column

In [15]:

query = """ 
SELECT *
FROM menu
WHERE price <= 0;
"""

menu = pd.read_sql_query(query, connection)

menu

,menu_item_id,item_name,category,price


#### There are no values at or below 0.

### Let's find the distinct values in category

In [16]:

query = """ 
SELECT DISTINCT(category)
FROM menu;
"""

menu = pd.read_sql_query(query, connection)

menu

,category
0,American
1,Asian
2,Mexican
3,Italian


#### There are four types of food categories: American, Asian, Mexican and Italian cuisine

### Let's find the least and most expensive items on the menu

In [17]:

query = """ 
SELECT item_name, category, MAX(price)
FROM menu;
"""

menu = pd.read_sql_query(query, connection)

menu

,item_name,category,MAX(price)
0,Shrimp Scampi,Italian,19.95


#### The most expensive item on the menu is the italian dish Shrimp Scampi at $19.95.

In [18]:

query = """ 
SELECT item_name, category, MIN(price)
FROM menu;
"""

menu = pd.read_sql_query(query, connection)

menu

,item_name,category,MIN(price)
0,Edamame,Asian,5.0


#### The least expensive item on the menu is the asian dish Edamame at $5.

### Since italian dishes make up the most expensive category, let's find the total number of different italian dishes

In [19]:
query = """ 
SELECT COUNT(*)
FROM menu
WHERE category LIKE '%Italian%';
"""

menu = pd.read_sql_query(query, connection)

menu

,COUNT(*)
0,9


In [20]:
query = """ 
SELECT item_name, category
FROM menu
WHERE category LIKE '%Italian%';
"""

menu = pd.read_sql_query(query, connection)

menu

,item_name,category
0,Spaghetti,Italian
1,Spaghetti & Meatballs,Italian
2,Fettuccine Alfredo,Italian
3,Meat Lasagna,Italian
4,Cheese Lasagna,Italian
5,Mushroom Ravioli,Italian
6,Shrimp Scampi,Italian
7,Chicken Parmesan,Italian
8,Eggplant Parmesan,Italian


#### There are 9 different italian dishes in this category

### Now lets see which Italian dishes are the least expensive

In [21]:
# Queries the least expensive dish under any category beginning and ending with italian 
query = """ 
SELECT item_name, category, MIN(price)
FROM menu
WHERE category LIKE '%Italian%';
"""

menu = pd.read_sql_query(query, connection)

menu

,item_name,category,MIN(price)
0,Spaghetti,Italian,14.5


#### Spaghetti is the least expensive italian dish at $14.5. Remember we already queried the most expensive dish which was Shrimp Scampi under the italian category

### Now lets find the amount of dishes per each category

In [22]:
# Queries the amount of dishes per category 
query = """ 
SELECT category, COUNT(item_name)
FROM menu
GROUP BY category
ORDER BY 1 DESC;
"""

menu = pd.read_sql_query(query, connection)

menu

,category,COUNT(item_name)
0,Mexican,9
1,Italian,9
2,Asian,8
3,American,6


#### Mexican and italian dishes are tied for the top spot with 9 dishes each. Asian dishes have the second highest ranking at number 8, with American being the lowest at 6.

### Let's look at the average dish price within each category

In [23]:
# Groups the average dish price by category and rounds to 2 decimal places
query = """ 
SELECT category, ROUND(AVG(price), 2) AS 'AVERAGE PRICE'
FROM menu
GROUP BY category
ORDER BY 2 DESC;
"""

menu = pd.read_sql_query(query, connection)

menu

,category,AVERAGE PRICE
0,Italian,16.75
1,Asian,13.48
2,Mexican,11.80
3,American,10.07


#### Italian dishes had the most expensive average price at 16.75, while American cuisine had the least expensive average at 10. 

# Objective 2
## Explore the orders table
### Your second objective is to better understand the orders table by finding the date range, the number of items within each order, and the orders with the highest number of items.

### Run SQL query and load results into Dataframe

In [24]:

query = """ 
SELECT *
FROM 'order';
"""

order = pd.read_sql_query(query, connection)

order

,order_details_id,order_id,order_date,order_time,item_id
0,1,1,1/1/23,11:38:36 AM,109.0
1,2,2,1/1/23,11:57:40 AM,108.0
2,3,2,1/1/23,11:57:40 AM,124.0
3,4,2,1/1/23,11:57:40 AM,117.0
4,5,2,1/1/23,11:57:40 AM,129.0
...,...,...,...,...,...
12229,12230,5369,3/31/23,10:05:04 PM,109.0
12230,12231,5369,3/31/23,10:05:04 PM,129.0
12231,12232,5369,3/31/23,10:05:04 PM,120.0
12232,12233,5369,3/31/23,10:05:04 PM,122.0


#### There are 5 columns: order_details_id (Unique id of an item in a order), order_id (ID of an order), order_date (Date an order was put in MM/DD/YY), order_time (Exact time an order was taken HH:MM:SS AM/PM) and item_id (Matches the menu item id from the menu table)
#### We can also see that the order_id contains certain numbers that include multiple item_id's

#### There are 12234 rows for the orders table 

### Let's look at the date range of the order table

In [25]:
query = """ 
SELECT MIN(order_date), MAX(order_date)
FROM 'order';
"""
order = pd.read_sql_query(query, connection)

order

,MIN(order_date),MAX(order_date)
0,1/1/23,3/9/23


#### The dates range from January 1, 2023 to March 9, 2023.

### Now we will query the number of orders made in that date range

In [26]:
query = """ 
SELECT COUNT(DISTINCT order_id)
FROM 'order';
"""
order = pd.read_sql_query(query, connection)

order

,COUNT(DISTINCT order_id)
0,5370


#### There are 5370 orders made during that date range

### We can also query the total number of items within each order

In [27]:
# Counts the number of items in the order table
query = """ 
SELECT COUNT(*)
FROM 'order';
"""
order = pd.read_sql_query(query, connection)

order

,COUNT(*)
0,12234


#### There are 12,234 items in the order table, compared to 5370 item_id's. This corresponds to the earlierst observation of the table, that shows there were instances of multiple item_id's for a singular order_id.

### Finding orders with the most number of items

In [28]:
query = """ 
SELECT order_id, COUNT(item_id) AS num_items
FROM 'order'
GROUP BY order_id
ORDER BY 2 DESC
LIMIT 10;
"""
order = pd.read_sql_query(query, connection)

order

,order_id,num_items
0,4305,14
1,3473,14
2,2675,14
3,1957,14
4,443,14
5,440,14
6,330,14
7,5200,13
8,5066,13
9,4836,13


#### There are 7 different order_id's 4305, 3473, 2675, 1975, 443, 440 and 330 that have a top number of 14

### Now we'll query how many orders had more than 12 items

In [29]:
#Creates a subquery within a new table named num_orders that queries the number of orders with more than 12 items
query = """ 
SELECT COUNT(*) FROM 

(SELECT order_id, COUNT(item_id) AS num_items
FROM 'order'
GROUP BY order_id
HAVING num_items > 12) AS num_orders;
"""
order = pd.read_sql_query(query, connection)

order

,COUNT(*)
0,20


#### There were 20 orders with more than 12 items

# Objective 3
## Analyze customer behavior
### Your final objective is to combine the items and orders tables, find the least and most ordered categories, and dive into the details of the highest spend orders.

### Lets combine the menu and order tables

### First we will take another look at the menu and order tables

In [30]:
query = """ 
SELECT *
FROM 'order'
LIMIT 10;
"""
order = pd.read_sql_query(query, connection)

order

,order_details_id,order_id,order_date,order_time,item_id
0,1,1,1/1/23,11:38:36 AM,109.0
1,2,2,1/1/23,11:57:40 AM,108.0
2,3,2,1/1/23,11:57:40 AM,124.0
3,4,2,1/1/23,11:57:40 AM,117.0
4,5,2,1/1/23,11:57:40 AM,129.0
5,6,2,1/1/23,11:57:40 AM,106.0
6,7,3,1/1/23,12:12:28 PM,117.0
7,8,3,1/1/23,12:12:28 PM,119.0
8,9,4,1/1/23,12:16:31 PM,117.0
9,10,5,1/1/23,12:21:30 PM,117.0


In [32]:
query = """ 
SELECT *
FROM menu
LIMIT 10;
"""
menu = pd.read_sql_query(query, connection)

menu

,menu_item_id,item_name,category,price
0,101,Hamburger,American,12.95
1,102,Cheeseburger,American,13.95
2,103,Hot Dog,American,9.00
3,104,Veggie Burger,American,10.50
4,105,Mac & Cheese,American,7.00
5,106,French Fries,American,7.00
6,107,Orange Chicken,Asian,16.50
7,108,Tofu Pad Thai,Asian,14.50
8,109,Korean Beef Bowl,Asian,17.95
9,110,Pork Ramen,Asian,17.95


#### Generally we want to keep the transaction data when we combine tables, which would be the order table, since it has the time and date transactions. A left join of the two tables would accomplish this.

In [36]:
# Creates an alias for the order table of od and mi for menu table. 
# Performs a left join of the order and menu table. 
query = """ 
SELECT *
FROM 'order' od
LEFT JOIN menu mi  
 ON od.item_id = mi.menu_item_id;
"""
order = pd.read_sql_query(query, connection)

order


,order_details_id,order_id,order_date,order_time,item_id,menu_item_id,item_name,category,price
0,1,1,1/1/23,11:38:36 AM,109.0,109.0,Korean Beef Bowl,Asian,17.95
1,2,2,1/1/23,11:57:40 AM,108.0,108.0,Tofu Pad Thai,Asian,14.50
2,3,2,1/1/23,11:57:40 AM,124.0,124.0,Spaghetti,Italian,14.50
3,4,2,1/1/23,11:57:40 AM,117.0,117.0,Chicken Burrito,Mexican,12.95
4,5,2,1/1/23,11:57:40 AM,129.0,129.0,Mushroom Ravioli,Italian,15.50
...,...,...,...,...,...,...,...,...,...
12229,12230,5369,3/31/23,10:05:04 PM,109.0,109.0,Korean Beef Bowl,Asian,17.95
12230,12231,5369,3/31/23,10:05:04 PM,129.0,129.0,Mushroom Ravioli,Italian,15.50
12231,12232,5369,3/31/23,10:05:04 PM,120.0,120.0,Steak Torta,Mexican,13.95
12232,12233,5369,3/31/23,10:05:04 PM,122.0,122.0,Chips & Salsa,Mexican,7.00


#### Are tables are combined and we now have 9 columns and 12234 rows

### Let's find the least and most ordered items and what categories they're in

In [53]:
# Keeps the previous join query and queries for the most to least ordered items along with their category 
query = """ 
SELECT item_name, category, COUNT(order_details_id) AS num_purchases
FROM 'order' od
LEFT JOIN menu mi  
 ON od.item_id = mi.menu_item_id
GROUP BY item_name, category
ORDER BY 3 DESC;
"""
order = pd.read_sql_query(query, connection)

order

,item_name,category,num_purchases
0,Hamburger,American,622
1,Edamame,Asian,620
2,Korean Beef Bowl,Asian,588
3,Cheeseburger,American,583
4,French Fries,American,571
5,Tofu Pad Thai,Asian,562
6,Steak Torta,Mexican,489
7,Spaghetti & Meatballs,Italian,470
8,Mac & Cheese,American,463
9,Chips & Salsa,Mexican,461


#### The most purchased item was the American hamburger at 622 purchases. The least purchased item at 123, were chicken tacos under the Mexican category.  Among the top 5 purchases are in the American and Asian. Many of the least purchased items were in the Mexican category.

### Top 5 orders that spent the most money

In [60]:
# Uses the left join function and queries the top 5 orders by total spent
query = """ 
SELECT order_id, item_name, category, SUM(price) AS total_spend
FROM 'order' od
LEFT JOIN menu mi  
 ON od.item_id = mi.menu_item_id
 GROUP BY 1
 ORDER BY 4 DESC
 LIMIT 5;
"""
order = pd.read_sql_query(query, connection)

order

,order_id,item_name,category,total_spend
0,440,Steak Tacos,Mexican,192.15
1,2075,Orange Chicken,Asian,191.05
2,1957,Orange Chicken,Asian,190.10
3,330,Orange Chicken,Asian,189.70
4,2675,Hamburger,American,185.10


#### 440, 2075, 1957, 330 and 2675 have the top 5 orders. 440 spent the most at 192.15 and order 2675 had the least at 185.10. 3 out of the five orders were the asian dish Orange Chicken.